<!---
Blue Brain Search is a text mining toolbox focused on scientific use cases.

Copyright (C) 2020  Blue Brain Project, EPFL.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU Lesser General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU Lesser General Public License for more details.

You should have received a copy of the GNU Lesser General Public License
along with this program. If not, see <https://www.gnu.org/licenses/>.
-->

# Demo of the Embedding REST API Server

To receive a sentence embedding proceed as follows:

- Wrap your query into a JSON file
- The JSON file should be of the following form:

```
    {
        "model": "<embedding model name>",
        "text": "<text>"
    }
```


- Send the JSON file to "/v1/embed"
- Receive a response as a CSV file

The current implementation handles one sentence per request. To compute embeddings for multiple sentences multiple queries need to be made.

The models currently available are:
- "USE" (Universal Sentence Embedding)
- "SBERT" (Sentence BERT)
- "SBIOBERT" (Sentence BioBERT)
- "BSV" (BioSentVec)

One can also get a welcome/help message by entering the plain server URL into your browser.

## Imports

In [ ]:
import io
import json
import requests

import numpy as np
import pandas as pd

## Helper Functions

In [ ]:
def make_request(request_url, payload):
    response = requests.get(url=request_url, json=payload)
        
    if response.ok:
        return response.text
    else:
        print("Response not OK :(")
        print("Status code:", response.status_code)
        print("Content:", response.text)
        return None

## Configure the Server

To find the correct port run the following command:

```bash
docker port embedding_server
```

if the output is

```
8080/tcp -> 0.0.0.0:32774
```

then the port variable should be set to 32774

In [ ]:
host = "<hostname>"
port = -1

In [ ]:
url = f"http://{host}:{port}/v1/embed"

## Create the request

In [ ]:
payload = {
    "model": "USE",
    "text": "This is a sample text."
}

## Send the request - response as JSON

In [ ]:
response = make_request(f"{url}/json", payload)

response_json = json.loads(response)
embedding = np.array(response_json["embedding"])
print("> Embedding shape:", embedding.shape)

## Send the request - response as CSV

In [ ]:
response = make_request(f"{url}/csv", payload)

string_stream = io.StringIO(response)
df_embedding = pd.read_csv(string_stream, header=None)
embedding = df_embedding.iloc[0].values
print("> Embedding shape:", embedding.shape)

## Send an invalid request

In [ ]:
make_request(f"{url}/microsoft_word", payload)